In [0]:
#Self Learning Chatbot

In [0]:
#Install Package NLTK(Natural Language Toolkit)
# ! It tells the notebook cell that this line is not a Python code, its a command line script.
!pip install nltk

In [0]:
#Install newspaper3k, it's a Article Scraping and Curation
!pip install newspaper3k

In [0]:
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

In [0]:
#Ignore Warning Messages
warnings.filterwarnings('ignore')

In [6]:
#Download Packages from NLTK
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [7]:
#Get Article URL
article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article.download()
article.parse()
article.nlp()
corpus = article.text

print(corpus)

Overview

Chronic kidney disease, also called chronic kidney failure, describes the gradual loss of kidney function. Your kidneys filter wastes and excess fluids from your blood, which are then excreted in your urine. When chronic kidney disease reaches an advanced stage, dangerous levels of fluid, electrolytes and wastes can build up in your body.

In the early stages of chronic kidney disease, you may have few signs or symptoms. Chronic kidney disease may not become apparent until your kidney function is significantly impaired.

Treatment for chronic kidney disease focuses on slowing the progression of the kidney damage, usually by controlling the underlying cause. Chronic kidney disease can progress to end-stage kidney failure, which is fatal without artificial filtering (dialysis) or a kidney transplant.

Chronic kidney disease care at Mayo Clinic

How kidneys work

Symptoms

Signs and symptoms of chronic kidney disease develop over time if kidney damage progresses slowly. Signs an

In [8]:
#Tokenization
text = corpus
#Converting text into list of sentences
sent_tokens = nltk.sent_tokenize(text)
print(sent_tokens)

['Overview\n\nChronic kidney disease, also called chronic kidney failure, describes the gradual loss of kidney function.', 'Your kidneys filter wastes and excess fluids from your blood, which are then excreted in your urine.', 'When chronic kidney disease reaches an advanced stage, dangerous levels of fluid, electrolytes and wastes can build up in your body.', 'In the early stages of chronic kidney disease, you may have few signs or symptoms.', 'Chronic kidney disease may not become apparent until your kidney function is significantly impaired.', 'Treatment for chronic kidney disease focuses on slowing the progression of the kidney damage, usually by controlling the underlying cause.', 'Chronic kidney disease can progress to end-stage kidney failure, which is fatal without artificial filtering (dialysis) or a kidney transplant.', 'Chronic kidney disease care at Mayo Clinic\n\nHow kidneys work\n\nSymptoms\n\nSigns and symptoms of chronic kidney disease develop over time if kidney damage

In [9]:
#Create Dictionary(key:value) to remove punctuations
remove_punct_dict = dict( (ord(punct), None) for punct in string.punctuation)
#Print Punctuation
print(string.punctuation)
#Print Dictionary
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [10]:
#Create a function to return list of limitised lower case after removing punctuations
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))  #remove_punct_dict to remove punctuation

print(LemNormalize(text))

['overview', 'chronic', 'kidney', 'disease', 'also', 'called', 'chronic', 'kidney', 'failure', 'describes', 'the', 'gradual', 'loss', 'of', 'kidney', 'function', 'your', 'kidneys', 'filter', 'wastes', 'and', 'excess', 'fluids', 'from', 'your', 'blood', 'which', 'are', 'then', 'excreted', 'in', 'your', 'urine', 'when', 'chronic', 'kidney', 'disease', 'reaches', 'an', 'advanced', 'stage', 'dangerous', 'levels', 'of', 'fluid', 'electrolytes', 'and', 'wastes', 'can', 'build', 'up', 'in', 'your', 'body', 'in', 'the', 'early', 'stages', 'of', 'chronic', 'kidney', 'disease', 'you', 'may', 'have', 'few', 'signs', 'or', 'symptoms', 'chronic', 'kidney', 'disease', 'may', 'not', 'become', 'apparent', 'until', 'your', 'kidney', 'function', 'is', 'significantly', 'impaired', 'treatment', 'for', 'chronic', 'kidney', 'disease', 'focuses', 'on', 'slowing', 'the', 'progression', 'of', 'the', 'kidney', 'damage', 'usually', 'by', 'controlling', 'the', 'underlying', 'cause', 'chronic', 'kidney', 'disease'

In [0]:
#Keyword Matching
#Greeting Input
GREETING_INPUTS = ["hi", "hello", "greetings", "wassup?", "hey"]
#Greeting Response
GREETING_RESPONSES = ["hi", "hello", "greetings", "how you doing?", "hey"]
#Response to User Greeting
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_RESPONSES)

In [0]:
#Generate the response
def response(user_response):


  #User response/query
  #user_response = 'What is chronic kidney disease'
  #Makes user response lower case 
  user_response = user_response.lower()

  #print(user_response)
  #set chatbot response in empty string
  robo_response = ''

  #append user response to sentence list
  sent_tokens.append(user_response)
  #print(sent_tokens)
  #Create TfidfVectorizer (term frequency–inverse document frequency) object
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words ='english')

  #Convert text to matrix TF-IDF feature
  tfidf = TfidfVec.fit_transform(sent_tokens)
  #print(tfidf)
  #Get the measure of similarity 
  vals = cosine_similarity(tfidf[-1], tfidf)
  #vals value between 0&1
  #print(vals)
  #Get the index of the most similar text/sentence to the user response
  idx = vals.argsort()[0][-2]

  #Reduce the dimensionality of vals
  flat = vals.flatten()
  flat.sort() #sort list in ascending order

  #Get most similar score to the users response 
  score = flat[-2]
  #print(score)

  #If variable 'score' is 0 then their is no text similar to the user response
  if(score == 0):
    robo_response = robo_response+"I am Sorry, I don't Understand Anything!"
  else:
    robo_response = robo_response+sent_tokens[idx]

  #Remove user response from sentence token list
  sent_tokens.remove(user_response)
  #print(robo_response)
  return robo_response




In [21]:
flag = True
print("Hello, I am DocBot for Chronic Kidney Disease. If you want to exit, type Bye! ")
while(flag == True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'bye'):
    if(user_response == 'thanks' or user_response == 'thank you'):  
      flag = False
      print("DocBot: You are Welcome!")
    else:
      if(greeting(user_response) != None):
        print("DocBot:"+ greeting(user_response))
      else:
        print("DocBot:" +response(user_response))
        
  else:
    flag = False
    print("DocBot: Will chat with you later!")

Hello, I am DocBot for Chronic Kidney Disease. If you want to exit, type Bye! 
hi
DocBot:hello
hello
DocBot:how you doing?
what is chronic kidney disease?
DocBot:Overview

Chronic kidney disease, also called chronic kidney failure, describes the gradual loss of kidney function.
what are potential complications?
DocBot:Potential complications may include:

Fluid retention, which could lead to swelling in your arms and legs, high blood pressure, or fluid in your lungs (pulmonary edema)

A sudden rise in potassium levels in your blood (hyperkalemia), which could impair your heart's ability to function and may be life-threatening

Heart and blood vessel (cardiovascular) disease

Weak bones and an increased risk of bone fractures

Anemia

Decreased sex drive, erectile dysfunction or reduced fertility

Damage to your central nervous system, which can cause difficulty concentrating, personality changes or seizures

Decreased immune response, which makes you more vulnerable to infection

Peric